In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("C:/Training/Academy/Statistics (Python)/Cases/Real Estate/Housing.csv")
dum_df = pd.get_dummies(df)
dum_df = dum_df.astype(float)
dum_df.head()

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_no,driveway_yes,recroom_no,recroom_yes,fullbase_no,fullbase_yes,gashw_no,gashw_yes,airco_no,airco_yes,prefarea_no,prefarea_yes
0,42000.0,5850.0,3.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
1,38500.0,4000.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,49500.0,3060.0,3.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,60500.0,6650.0,3.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,61000.0,6360.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [3]:
dum_df.shape

(546, 18)

In [4]:
X = dum_df.iloc[:,1:]
y = dum_df['price']
scaler = MinMaxScaler()
scalerY = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=23)
y_train = scalerY.fit_transform(y_train.values.reshape(-1, 1))
X_train = scaler.fit_transform(X_train)
#y_test = scalerY.transform(y_test.values.reshape(-1, 1))
X_test = scaler.transform(X_test)

In [5]:
X_torch = torch.from_numpy(X_train)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([382, 17])
torch.Size([382, 1])


In [6]:
torch.manual_seed(23)
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_train.shape[1], out_features=8),
                      nn.ReLU(),
                      nn.Linear(8,6),
                      nn.ReLU(),
                      nn.Linear(6,4),
                      nn.ReLU(),
                      nn.Linear(4, 3),
                      nn.ReLU(),
                      nn.Linear(3,1),
                      nn.ReLU())

In [7]:
criterion = torch.nn.MSELoss()
# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    weight_decay: 0
)

Initail Weights

In [8]:
model[0].weight

Parameter containing:
tensor([[-0.0348, -0.1024, -0.0376, -0.0693,  0.2220, -0.1892, -0.1002,  0.2040,
          0.0425, -0.1795,  0.0838, -0.1927,  0.1395,  0.0262, -0.0168, -0.1304,
         -0.1351],
        [-0.0801, -0.0223, -0.1203,  0.0635, -0.1598, -0.0911, -0.1467,  0.0226,
         -0.2322,  0.1964,  0.1671,  0.2100, -0.0994, -0.0110, -0.0103, -0.0757,
          0.0840],
        [ 0.0773, -0.1514, -0.0220, -0.1917,  0.1025,  0.1314,  0.0249, -0.2034,
         -0.0956,  0.1424, -0.1056,  0.0333, -0.1769, -0.0376, -0.1482, -0.2042,
         -0.1338],
        [-0.0915,  0.2080,  0.2071,  0.0976,  0.1352,  0.0800, -0.1087,  0.1077,
         -0.0120, -0.1647, -0.0381,  0.1310, -0.0318,  0.0107,  0.0718,  0.0846,
         -0.1852],
        [-0.1370,  0.1214,  0.1692, -0.0745,  0.1400,  0.0627,  0.1856,  0.1022,
         -0.0120,  0.1870, -0.1059, -0.0271,  0.0739,  0.1360, -0.2207, -0.0089,
          0.1923],
        [ 0.2223,  0.1324,  0.1182,  0.0637, -0.1643,  0.0434, -0.0968,  

In [9]:
model

Sequential(
  (0): Linear(in_features=17, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=6, bias=True)
  (3): ReLU()
  (4): Linear(in_features=6, out_features=4, bias=True)
  (5): ReLU()
  (6): Linear(in_features=4, out_features=3, bias=True)
  (7): ReLU()
  (8): Linear(in_features=3, out_features=1, bias=True)
  (9): ReLU()
)

In [10]:
y_pred = model(X_torch.float())
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([382, 1])
torch.Size([382, 1])


In [11]:
# Gradient Descent

for epoch in np.arange(0,1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(X_torch.float())

   # Compute and print loss
   loss = criterion(y_pred, y_torch.float())
   #print('epoch: ', epoch+1,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
   if epoch % 100 == 0:
       print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.02720540389418602
epoch:  101  loss:  0.027075545862317085
epoch:  201  loss:  0.02657013013958931
epoch:  301  loss:  0.009478576481342316
epoch:  401  loss:  0.008017164655029774
epoch:  501  loss:  0.007675302680581808
epoch:  601  loss:  0.0076583195477724075
epoch:  701  loss:  0.007691293489187956
epoch:  801  loss:  0.007676278240978718
epoch:  901  loss:  0.007794701494276524


In [12]:
#model = model.eval()
X_torch_test = torch.from_numpy(X_test)
y_pred = model(X_torch_test.float())
y_pred = y_pred.detach().numpy()
y_pred = y_pred.reshape(y_test.shape[0],)
y_pred[:5]

array([0.51723206, 0.13676906, 0.21847744, 0.13676906, 0.13676906],
      dtype=float32)

In [13]:
y_pred = y_pred.reshape(-1,1)
y_pred_orig = scalerY.inverse_transform(y_pred)
y_pred_orig[:5]

array([[110343.29 ],
       [ 47566.895],
       [ 61048.777],
       [ 47566.895],
       [ 47566.895]], dtype=float32)

In [14]:
y_test.iloc[:5]

231    87000.0
155    60000.0
424    65500.0
268    54000.0
19     45000.0
Name: price, dtype: float64

### Test Set R2 Score

In [15]:
print(r2_score(y_test,y_pred_orig))

0.5913178540683988


### Test Set MSE

In [16]:
mse(y_test,y_pred_orig)

263530852.70905462